In [7]:
import numpy as np

def softmax(x):
    e_x = np.exp(x-np.max(x))
#     e_x = np.exp(x)
    return e_x / e_x.sum(axis=0)

def sigmoid(x):
    return 1/(1+np.exp(-x))

def initialize_adam(parameters):
    L = len(parameters)/2
    v = {}
    s = {}
    for l in range(L):
        v['dW'+str(l+1)] = np.zeros(parameters['W'+str(l+1)].shape)
        v['db'+str(l+1)] = np.zeros(parameters['b'+str(l+1)].shape)
        s['dW'+str(l+1)] = np.zeros(parameters['W'+str(l+1)].shape)
        s['db'+str(l+1)] = np.zeros(parameters['b'+str(l+1)].shape)
    return v,s

def update_parameters_with_adam(parameters,grads,v,s,t,
                                learning_rate=0.01,beta1=0.9,beta2=0.999,epsilon=1e-8):
    # parameters['W'+str(l)] = W1
    # grads['dW'+str(l)] = dW1
    L = len(parameters)//2
    v_corrected = {}
    s_corrected = {}
    
    for l in range(L):
        v['dW'+str(l+1)] = beta1*v['dW'+str(l+1)]+(1-beta1)*grads['dW'+str(l+1)]
        v['db'+str(l+1)] = beta1*v['db'+str(l+1)]+(1-beta1)*grads['db'+str(l+1)]
        s['dW'+str(l+1)] = beta2*s['dW'+str(l+1)]+(1-beta2)*(grads['dW'+str(l+1)]**2)
        s['db'+str(l+1)] = beta2*s['db'+str(l+1)]+(1-beta2)*(grads['db'+str(l+1)]**2)
        
        v_corrected['dW'+str(l+1)] = v['dW'+str(l+1)]/(1-beta1**t)
        v_corrected['db'+str(l+1)] = v['db'+str(l+1)]/(1-beta1**t)
        s_corrected['dW'+str(l+1)] = s['dW'+str(l+1)]/(1-beta2**t)
        s_corrected['db'+str(l+1)] = s['db'+str(l+1)]/(1-beta2**t)
        
        parameters['W'+str(l+1)] = parameters['W'+str(l+1)]-learning_rate*v_corrected['dW'+str(l+1)]/np.sqrt(s['dW'+str(l+1)]+epsilon)
        parameters['b'+str(l+1)] = parameters['b'+str(l+1)]-learning_rate*v_corrected['db'+str(l+1)]/np.sqrt(s['db'+str(l+1)]+epsilon)
    return parameters, v, s

def rnn_cell_forward(xt, a_prev, parameters):
    Wax = parameters['Wax']
    Waa = parameters['Waa']
    ba = parameters['ba']
    by = parameters['by']
    
    a_next = np.tanh(np.dot(Waa,a_prev) + np.dot(Waa, xt) + ba)
    yt_pred = softmax(np.dot(Way, a_next) + by)
    cache = (a_next, a_prev, xt, parameters)
    return a_next, yt_pred, cache

def rnn_forward(x, a0, parameters):
    caches = []
    n_x, m, T_x = x.shape
    n_y, n_a = parameters['Wya'].shape
    
    a = np.zeros((n_a,m,T_x))
    y_pred = np.zeros((n_y, m, T_x))
    a_next = a0
    
    for t in range(T_x):
        a_next, yt_pred, cache = rnn_cell_forward(x[:,:,t],a_next,parameters)
        a[:,:,t] = a_next
        y_pred[:,:,t] = yt_pred
        caches.append(cache)
    caches = (caches, x)
    return a, y_pred, caches

def lstm_cell_forward(xt, a_prev, c_prev, parameters):
    # weights matrix of the forget gate
    Wf = parameters['Wf']
    bf = parameters['bf']
    # weights matrix og the update gate
    Wi = parameters['Wi']
    bi = parameters['bi']
    # weights matrix of the first tanh
    Wc = parameters['Wc']
    bc = parameters['bc']
    # weights matrix of the output gate
    Wo = parameters['Wo']
    bo = parameters['bo']
    # weights matrix relating the hidden-state to the output
    Wy = parameters['Wy']
    by = parameters['by']
    
    n_x, m = xt.shape
    n_y, n_a = Wy.shape
    
    concat = np.zeros((n_x+n_a, m))
    concat[:n_a, :] = a_prev
    concat[n_a:, :] = xt
    
    ft = sigmoid(np.dot(Wf, concat)+bf)
    it = sigmoid(np.dot(Wi, concat)+bi)
    cct = np.tanh(np.dot(Wc, concat)+bc)
    c_next = ft*c_prev + it*cct
    ot = sigmoid(np.dot(Wo, concat)+bo)
    a_next = ot*np.tanh(c_next)
    
    yt_pred = softmax(np.dot(Wy, a_next)+by)
    caches = (a_next,c_next,a_prev,c_prev,ft,it,cct,ot,xt,parameters)
    return a_next, c_next, yt_pred, cache

deff lstm_forward(x, a0, parameters):
    caches = []
    n_x, m, T_x = x.shape
    n_y, n_a = parameters['Wy'].shape
    
    a = np.zeros((n_a, m, T_X))
    c = np.zeros((n_a, m, T_x))
    y = np.zeros((n_y, m, T_x))
    
    a_next = a0
    c_next = np.zeros((n_a, m))
    
    for t in range(T_x):
        a_next,c_next,yt, cache = lstm_cell_forward(x[:,:,t],a_next,c_next,parameters)
        a[:,:,t] = a_next
        y[:,:,t] = yt
        c[:,:,t] = c_next
        cachess.append(cache)
    caches = (caches, x)
    return a,y,c,caches

[0.09003057 0.24472847 0.66524096]
